In [1]:
import torch
import torch.utils.data as data
import pandas as pd
import matplotlib.pyplot as plt
from src.error_measures import get_accuracy, get_error_measures, collect_stats, print_average_stats, gather_predictions
from src.models import MultiLayerANN, SynapticSNN, LeakySNN, DoubleLeakySNN
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Data

In [3]:
data12 = pd.read_csv("../data/processed/target12.csv")
data12.drop(columns=["weekday"], inplace=True)

In [4]:
all_data = data.TensorDataset(torch.from_numpy((data12.values[:,:-1] - data12.values[:,:-1].min(0)) / data12.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data12.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.8 * len(all_data)), round(0.1 * len(all_data)), round(0.1 * len(all_data))))

In [5]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

### Simple regressor

In [6]:
model1 = MultiLayerANN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model1)

SimpleANN(
  (lin1): Linear(in_features=8, out_features=3500, bias=True)
  (tanh): Tanh()
  (lin2): Linear(in_features=3500, out_features=3500, bias=True)
  (relu): ReLU()
  (lin3): Linear(in_features=3500, out_features=1, bias=True)
)


In [7]:
training_loop(model1, train_loader, valid_loader, device, num_epochs=100, lr=0.003, validation=True)

  1%|          | 1/100 [00:04<07:42,  4.67s/it]

MAE: 8.5964, MSE:138.7623,
RMSE: 11.7797, IA: 0.7844, MAPE: 59.7695%


  2%|▏         | 2/100 [00:06<04:44,  2.91s/it]

MAE: 8.2042, MSE:144.9369,
RMSE: 12.0390, IA: 0.7464, MAPE: 51.1733%


  3%|▎         | 3/100 [00:08<03:49,  2.36s/it]

...

MAE: 6.4774, MSE:86.2822,
RMSE: 9.2888, IA: 0.8892, MAPE: 35.9478%


 99%|█████████▉| 99/100 [02:48<00:01,  1.65s/it]

MAE: 7.0966, MSE:91.6264,
RMSE: 9.5722, IA: 0.8956, MAPE: 46.5617%


100%|██████████| 100/100 [02:49<00:00,  1.70s/it]

MAE: 6.6195, MSE:85.3354,
RMSE: 9.2377, IA: 0.8993, MAPE: 40.9363%
Accuracy on validation dataset: 47.74%


In [8]:
print(f'Accuracy on test set: {get_accuracy(model1, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model1, test_loader, device=device, print_e=True)

Accuracy on test set: 45.540000000000006%
MAE: 6.2182, MSE:70.0225,
RMSE: 8.3679, IA: 0.8697, MAPE: 43.0139%


(6.218238733389424,
 70.02248361721247,
 8.367943810591253,
 0.8697036546021201,
 43.01386963200978)

### SNN with Leaky layer

In [9]:
model2 = LeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1, num_steps=25)
print(model2)

LeakySNN(
  (fc1): Linear(in_features=8, out_features=3500, bias=True)
  (lif): Leaky()
  (fc2): Linear(in_features=3500, out_features=1, bias=True)
)


In [10]:
training_loop(model2, train_loader, valid_loader, device, num_epochs=25, lr=0.001, validation=True)

  4%|▍         | 1/25 [00:05<02:17,  5.74s/it]

MAE: 7.9596, MSE:135.3226,
RMSE: 11.6328, IA: 0.7911, MAPE: 48.9932%


  8%|▊         | 2/25 [00:11<02:11,  5.70s/it]

MAE: 8.1084, MSE:142.2802,
RMSE: 11.9281, IA: 0.7991, MAPE: 43.1768%


 12%|█▏        | 3/25 [00:17<02:05,  5.72s/it]

...

MAE: 6.8767, MSE:90.3762,
RMSE: 9.5066, IA: 0.8793, MAPE: 44.8334%


 96%|█████████▌| 24/25 [02:15<00:05,  5.51s/it]

MAE: 6.7413, MSE:89.1360,
RMSE: 9.4412, IA: 0.8875, MAPE: 43.4872%


100%|██████████| 25/25 [02:21<00:00,  5.64s/it]

MAE: 7.1766, MSE:91.3375,
RMSE: 9.5571, IA: 0.8907, MAPE: 50.4521%
Accuracy on validation dataset: 44.73%


In [11]:
print(f'Accuracy on test set: {get_accuracy(model2, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model2, test_loader, device=device, print_e=True)

Accuracy on test set: 41.6%
MAE: 6.7399, MSE:74.5174,
RMSE: 8.6323, IA: 0.8647, MAPE: 51.8016%


(6.739942054276118,
 74.51742005584002,
 8.632347308573724,
 0.8646636169928696,
 51.8015526732084)

### SNN with Synaptic layer

In [12]:
model3 = SynapticSNN(num_inputs=8, num_hidden=3500, num_outputs=1, num_steps=50)
print(model3)

SynapticSNN(
  (fc1): Linear(in_features=8, out_features=3500, bias=True)
  (lif): Synaptic()
  (fc2): Linear(in_features=3500, out_features=1, bias=True)
)


In [13]:
training_loop(model3, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=True)

  4%|▍         | 1/25 [00:11<04:27, 11.13s/it]

MAE: 9.8072, MSE:176.2153,
RMSE: 13.2746, IA: 0.7421, MAPE: 49.8143%


  8%|▊         | 2/25 [00:22<04:24, 11.50s/it]

MAE: 7.5273, MSE:115.3310,
RMSE: 10.7392, IA: 0.8072, MAPE: 42.3166%


 12%|█▏        | 3/25 [00:34<04:14, 11.58s/it]

...

MAE: 6.2223, MSE:72.8626,
RMSE: 8.5360, IA: 0.8973, MAPE: 42.0917%


 96%|█████████▌| 24/25 [04:32<00:11, 11.49s/it]

MAE: 6.1031, MSE:72.4878,
RMSE: 8.5140, IA: 0.8937, MAPE: 37.7609%


100%|██████████| 25/25 [04:43<00:00, 11.35s/it]

MAE: 9.0045, MSE:121.9081,
RMSE: 11.0412, IA: 0.8434, MAPE: 70.2105%
Accuracy on validation dataset: 34.300000000000004%


In [14]:
print(f'Accuracy on test set: {get_accuracy(model3, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model3, test_loader, device=device, print_e=True)

Accuracy on test set: 30.48%
MAE: 9.3394, MSE:128.4582,
RMSE: 11.3339, IA: 0.8386, MAPE: 74.3396%


(9.339397490922435,
 128.4582106092612,
 11.33394064786212,
 0.8385622808506112,
 74.33962311672066)

### SNN with two Leaky layers

In [15]:
model4 = DoubleLeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model4)

DoubleLeakySNN(
  (lin1): Linear(in_features=8, out_features=3500, bias=True)
  (lif1): Leaky()
  (lif2): Leaky()
  (lin2): Linear(in_features=3500, out_features=1, bias=True)
)


In [16]:
training_loop(model4, train_loader, valid_loader, device, num_epochs=25, lr=0.002, validation=True)

  4%|▍         | 1/25 [00:10<04:06, 10.28s/it]

MAE: 20.7912, MSE:648.0209,
RMSE: 25.4563, IA: 0.4117, MAPE: 96.9298%


  8%|▊         | 2/25 [00:21<04:03, 10.60s/it]

MAE: 9.2240, MSE:158.7891,
RMSE: 12.6012, IA: 0.6237, MAPE: 61.6509%


 12%|█▏        | 3/25 [00:31<03:52, 10.57s/it]

...

 96%|█████████▌| 24/25 [04:06<00:10, 10.53s/it]

MAE: 7.6525, MSE:118.2745,
RMSE: 10.8754, IA: 0.7762, MAPE: 47.5426%


100%|██████████| 25/25 [04:17<00:00, 10.29s/it]

MAE: 7.5988, MSE:115.5370,
RMSE: 10.7488, IA: 0.7869, MAPE: 47.7704%


Accuracy on validation dataset: 41.14%


In [17]:
print(f'Accuracy on test set: {get_accuracy(model4, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model4, test_loader, device=device, print_e=True)

Accuracy on test set: 41.6%
MAE: 7.2979, MSE:105.2202,
RMSE: 10.2577, IA: 0.8092, MAPE: 50.2140%


(7.2979026539715255,
 105.22018341455497,
 10.257688989950658,
 0.809150441530647,
 50.21399957549717)

### Repeated training on random datasets

In [5]:
err_model1 = [0., 0., 0., 0., 0., 0.]
err_model2 = [0., 0., 0., 0., 0., 0.]
err_model3 = [0., 0., 0., 0., 0., 0.]
err_model4 = [0., 0., 0., 0., 0., 0.]

In [6]:
trials = 2
for i in range(trials):
    # Prepare data loaders.
    train_dataset, test_dataset = torch.utils.data.random_split(all_data, (round(0.8 * len(all_data)), round(0.2 * len(all_data))))
    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
    test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

    # Prepare models.
    model1 = MultiLayerANN(num_inputs=8, num_hidden=4000, num_outputs=1)
    model2 = LeakySNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=25)
    model3 = SynapticSNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=50)
    model4 = DoubleLeakySNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=20)

    # Train models.
    print(f'------------------------------------\nTraining {i+1}')
    training_loop(model1, train_loader, test_loader, device, num_epochs=100, lr=0.003, validation=False) #90
    training_loop(model2, train_loader, test_loader, device, num_epochs=60, lr=0.001, validation=False) #50
    training_loop(model3, train_loader, test_loader, device, num_epochs=30, lr=0.0001, validation=False) #25
    training_loop(model4, train_loader, test_loader, device, num_epochs=40, lr=0.001, validation=False) #35

    # Gather results.
    collect_stats(model1, err_model1, test_loader, device)
    collect_stats(model2, err_model2, test_loader, device)
    collect_stats(model3, err_model3, test_loader, device)
    collect_stats(model4, err_model4, test_loader, device)

------------------------------------
Training 1


100%|██████████| 100/100 [03:30<00:00,  2.10s/it]


Accuracy on validation dataset: 42.0%


100%|██████████| 60/60 [05:12<00:00,  5.21s/it]


Accuracy on validation dataset: 50.63999999999999%


...

Accuracy on validation dataset: 47.68%


100%|██████████| 40/40 [05:17<00:00,  7.94s/it]


Accuracy on validation dataset: 48.84%
------------------------------------
Training 2


 22%|██▏       | 22/100 [00:47<02:49,  2.17s/it]


KeyboardInterrupt: 

In [10]:
# Print average results
print('------Simple ANN ------')
print_average_stats(err_model1, trials)
print('------LIF SNN ------')
print_average_stats(err_model2, trials)
print('------Synaptic SNN ------')
print_average_stats(err_model3, trials)
print('------Double LIF SNN ------')
print_average_stats(err_model4, trials)

------Simple ANN ------
MAE: 5.733594991551628, MSE:63.580033061411996,
RMSE: 7.967944587224418, IA: 0.9164841564885322,
MAPE: 38.03590761235476 %, acc: 51.449999999999996 %

------LIF SNN ------
MAE: 6.2430069344668615, MSE:69.54496934137244,
RMSE: 8.339354083040947, IA: 0.9103193725694927,
MAPE: 44.6369944580173 %, acc: 48.089999999999996 %

------Synaptic SNN ------
MAE: 9.264562545325347, MSE:135.060263711052,
RMSE: 11.233460025005105, IA: 0.8579632085418573,
MAPE: 68.35216490927189 %, acc: 33.75 %

------Double LIF SNN ------
MAE: 6.685834302134199, MSE:80.7118603840073,
RMSE: 8.983535278879645, IA: 0.8816772396448564,
MAPE: 48.23070322089788 %, acc: 45.655 %

